## Create table 3

This code reads the output files from the models and compiles table 3.

The model names in the paper map into the code according to: 

   * Benchmark:  \01Benchmark
   * Sunk Cost:   \02Sunkcost
   * Sunk-cost High: \03SunkHigh
   * No cost: \04NoCostSluggish
   * Reentry:  \05ReEntry
   * Search:  \06Search
   * Starters: \07Starter
   * Sunk-cost Iceberg: 08SunkIceberg

In [1]:
import pandas as pd
import numpy as np
import json

### Model parameters

The code that creates table 2 saves the parameter values from all the models into `params.json`. 

In [2]:
params = json.load(open('params.json'))

### Hardcoded parameters

These should be double checked after calibration

In [3]:
# We always consider a tariff change from 10 percent to zero.
tau0 = 1.1
tau1 = 1.0

# In the sunk-iceberg model (and only in the sunk-iceberg model) the trade costs fall with the liberalization. 
# The initial values are in the params dict. These are the new ones.
xiL_ice = 1.31865253
xiH_ice = 1.31865253

# For all models except sunk-iceberg, the pre- and post-liberalization values are the same.
for k in params:
    params[k]['xiL_new'] = params[k]['xiL']
    params[k]['xiH_new'] = params[k]['xiH']
    
params['sunkice']['xiL_new'] = xiL_ice
params['sunkice']['xiH_new'] = xiH_ice


### Function definitions

In [4]:
# Period utility
def u(c, sigma):
    return (c**(1-sigma))/(1-sigma)

This function reads in the transition paths from the model.out files. It generates a few new variables, too.

In [5]:
def read_transition(file_path, params, model):
    
    alpha = params['alpha']
    alphax = params['alpm']
    theta = params['theta']
    
    fp =  open(file_path)
    buffer = fp.readlines()
    buffer = [b.strip() for b in  buffer]
    buffer = [b for b in buffer if len(b)>0]

    # The transition data start with the header, which starts with 'Period'
    trans = []
    saveit = 0
    for i, elm in enumerate(buffer):
        if elm.startswith('Period'):
            saveit = 1
            linelen = len(elm.split())
         
        # There may be crud after the transition
        if (saveit == 1) and (len(elm.split())==linelen):
            trans.append(elm.split())

    trans = pd.DataFrame(trans[1:], columns=trans[0], dtype=float)
    trans = trans.set_index('Period')
    
    # Compute a few more variables. 
    # Compute alpha K/L
    trans['alphaKL'] = (trans['K0']/trans['LP'])**alpha
    
    # a seris for tau
    trans['tau'] = pd.Series([tau1]*len(trans))
    trans.loc[0,'tau'] = tau0
    
    # xil and xih change in the sunk-iceberg model, so create series of trade costs.
    trans['xiL'] = params['xiL_new']
    trans['xiH'] = params['xiH_new']
    trans.loc[0,'xiL'] = params['xiL']
    trans.loc[0,'xiH'] = params['xiH']
    
    
    # aggregate export intensity
    wtdcost = trans['xiL']**(1-theta)*trans['IntL'] + trans['xiH']**(1-theta)*trans['IntH']
    trans['eei'] = trans['tau']**(-theta)*wtdcost / (trans['IntL'] + trans['IntH'] + trans['tau']**(-theta)*wtdcost) 
        
    # export share
    trans['exs'] = 1/(1+trans['tau']*np.power(trans['IMD'], -1)) 
    
    # iceberg costs
    trans['ice'] = (trans['eei']/(1-trans['eei']) *trans['tau']**(theta))**(1/(1-theta)) -1
    
    # S
    trans['s'] = (trans['IntT'] + trans['tau']**(1-theta)*wtdcost) / (trans['IntT'] + trans['tau']**(-theta)*wtdcost)
    trans['s_adj'] = trans['s'] - (alphax*(theta-1)/theta)
    
    # In the no cost model, thA is the relevant parameter in these calculations. 
    if model=='nocost':
        thA = params['thA']
        wtdcost = trans['xiL']**(1-thA)*trans['IntL'] + trans['xiH']**(1-thA)*trans['IntH']
                
        trans['eei'] = trans['tau']**(-thA)*wtdcost / (trans['IntL'] + trans['IntH'] + trans['tau']**(-thA)*wtdcost)
        
        trans['ice'] = (trans['eei']/(1-trans['eei']) *trans['tau']**(thA))**(1/(1-thA)) -1
               
        trans['s'] = (trans['IntT'] + trans['tau']**(1-thA)*trans['xiH']**(1-thA)*trans['IntT']) / (trans['IntT'] + trans['tau']**(-thA)*trans['xiH']**(1-thA)*trans['IntT'])
        
        trans['s_adj'] = trans['s'] - (alphax*(thA-1)/thA)
    
    return trans

### Load the transition path data

Read in the transition path data for each model.

In [6]:
paths = ['../Model code/01Benchmark/Benchamrk01.out', '../Model code/02SunkCost/SunkCost01.out',
        '../Model code/03SunkHigh/SunkHigh01.out', '../Model code/08SunkIceberg/Sunk_Iceberg01.out', 
        '../Model code/05ReEntry/Reentry01.out', '../Model code/06Search/Search01.out',
        '../Model code/07Starter/Starter01.out', '../Model code/04NoCostSluggish/NoCost01.out']

models = ['bench', 'sunk', 'sunkhigh', 'sunkice', 'reentry', 'search', 'starter', 'nocost']
trans = {}

for m, p in zip(models, paths):
    trans[m] = read_transition(p, params[m], m)


### Compute the steady-state to steady-state changes

In [7]:
def compute_ss_changes(trans, params, model):
    theta = params['theta']
    alphax = params['alpm']
    
    ss_change = np.log(trans.loc[303]/trans.loc[0])*100
    
    # psid hat normalzed
    ss_change.loc['psidhat_norm'] = (ss_change['IntT'] - ss_change['NT'])/(theta-1)/(1-alphax)

    # lmbda hat normalized
    ss_change.loc['lambdahat_norm'] = -ss_change['lambda']/(theta-1)/(1-alphax)

    # N hat normalized
    ss_change['nhat_norm'] = ss_change['NT']/(theta-1)/(1-alphax)

    # elasticity
    ss_change['elast'] = ss_change['IMD']/np.log(1.1)/100
    
    # exporter premium
    ss_change['prem'] = ss_change['exs'] - ss_change['eei'] - ss_change['n_X']
    
    # In the no-cost model, thA is the relevant parameter. 
    if model == 'nocost':
        thA = params['thA']
        ss_change.loc['psidhat_norm'] = (ss_change['IntT'] - ss_change['NT'])/(thA-1)/(1-alphax)
        ss_change.loc['lambdahat_norm'] = -ss_change['lambda']/(thA-1)/(1-alphax)
        ss_change['nhat_norm'] = ss_change['NT']/(thA-1)/(1-alphax)
    
    return ss_change

In [8]:
ss_changes = {}

for m in models:
    ss_changes[m] = compute_ss_changes(trans[m], params[m], m)

C:\Users\kimru\anaconda3\lib\site-packages\pandas\core\series.py:679: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


### Discounted changes

The discounted changes are the sum of the period t growth rate multiplied by beta to the t-1, plus an adjustment that assumes we are in the steady state in the last period of our computed transition. 

In [9]:
def discounted_growth(var, betat, T, beta):
    # The transition growth rates weighted by beta^t
    temp = (np.log(var/var.iloc[0]) * betat).sum() 
    # The adjustment for the new steady state
    return (temp + np.log(var.iloc[-1]/var.iloc[0])/(1-beta)*beta**(T) )*(1-beta)*100

In [10]:
def compute_dis_changes(trans, params, model):
    '''Pass this the DataFrame of transition paths from a model and return the discounted growth rates.'''
    res = {}
    
    beta = params['beta']
    theta = params['theta']
    alphax = params['alpm']
    sigma = params['sigma']
    T = trans.shape[0]-1
    
    # Column of beta^t = 0, 1, beta, beta^2,...
    trans['betat'] = pd.Series([beta**(t-1) for t in range(0, T+1)], name='betat')
    trans.loc[0, 'betat'] = 0
        
    # Discounted Y hat
    res['dy'] = discounted_growth(trans['Y'], trans['betat'], T, beta)
    
    # Discounted S hat - ax(theta-1)/theta
    res['dsadj'] = discounted_growth(trans['s_adj'], trans['betat'], T, beta)
    
    # Discounted Lp hat
    res['dlp'] = discounted_growth(trans['LP'], trans['betat'], T, beta)
        
    # Discounted alpha K/L
    res['dkl'] = discounted_growth(trans['alphaKL'], trans['betat'], T, beta)
        
    # Discounted N
    res['dn'] = discounted_growth(trans['NT'], trans['betat'], T, beta)/(theta-1)/(1-alphax)
    
    if model == 'nocost':
        thA = params['thA']
        res['dn'] = discounted_growth(trans['NT'], trans['betat'], T, beta)/(thA-1)/(1-alphax)
        
    # Discounted psid
    res['dpsid'] = discounted_growth(trans['IntT']/trans['NT'], trans['betat'], T, beta)/(theta-1)/(1-alphax)
    
    if model == 'nocost':
        thA = params['thA']
        res['dpsid'] = discounted_growth(trans['IntT']/trans['NT'], trans['betat'], T, beta)/(thA-1)/(1-alphax)
    
    # Discounted lambda
    res['dlam'] = -discounted_growth(trans['lambda'], trans['betat'], T, beta)/(theta-1)/(1-alphax)
    
    if model == 'nocost':
        thA = params['thA']
        res['dlam'] = -discounted_growth(trans['lambda'], trans['betat'], T, beta)/(thA-1)/(1-alphax)
        
    # Elasticity
    res['delas'] = (np.log(trans['IMD']/trans.loc[0, 'IMD'])/np.log(1.1) * trans['betat']).sum()
    res['delas'] = (res['delas'] + (np.log(trans.loc[T, 'IMD']/trans.loc[0, 'IMD'])/np.log(1.1))/(1-beta)*beta**(T))*(1-beta)
    
    # Welfare ---------------------------------------------------------------------------
    # Welfare at time zero is u(C_0) * 1/(1-beta) since we are in steady state
    W0 = u(trans.loc[0,'C'], sigma)/(1-beta)
    
    # Sum up welfare in our transition
    trans['u'] = trans['C'].apply(u, args=(sigma,) )
    W1 = (trans['betat'] * trans['u']).sum()
    
    # Assume that consumption has converged to the new steady state
    final_cons = trans.loc[T, 'u'] 
    W1 = W1 + final_cons/(1-beta) * beta**(T+1)
    
    res['dwelfare'] = -np.log(W1/W0)*100
    
    return  pd.Series(res)


In [11]:
dis_changes = {}
for m in models:
    dis_changes[m] = compute_dis_changes(trans[m], params[m], m)

### Create table 3

These next two function print out a row of the table. The `format_row_dis` is only different in that it puts square brackets around the values and adds some extra space after the line.

In [12]:
def format_row(symbol, sym_name, data, models, fmt):
    t = symbol + ' & '
    
    for c in models:
        fstr = '{0:' + fmt + '}'
        if data[c][sym_name] == '--':
            t = t + '--' + ' & '
        else:
            t = t + fstr.format(float(data[c][sym_name])) + ' & '

    t = t[:-2] + '\\\ \n'
    return t

In [13]:
def format_row_dis(symbol, sym_name, data, models, fmt):
    t = symbol + ' & '
    
    for c in models:
        fstr = '[{0:' + fmt + '}]'
        if data[c][sym_name] == '--':
            t = t + '--' + ' & '
        else:
            t = t + fstr.format(float(data[c][sym_name])) + ' & '

    t = t[:-2] + '\\\ [1ex] \n'
    return t

In [14]:
fw = open('table_3.tex', 'w')

fw.write(r'\documentclass[12pt,leqno]{article}' + '\n')
fw.write(r'\usepackage{booktabs}' + '\n')
fw.write(r'\usepackage{siunitx}' + '\n')
fw.write(r'\usepackage[left=0.5in,right=0.5in,top=1in,bottom=1in]{geometry}' + '\n')

fw.write(r'\sisetup{input-decimal-markers = .,input-ignore = {,}, parse-numbers=false, group-separator={,}, group-four-digits = false}'+'\n')
fw.write(r'\begin{document}' + '\n')

fw.write(r'\begin{table}[tbp]' + '\n')
fw.write(r'\small \centering')
fw.write(r'\begin{tabular}{lSSSSSSSS}' + '\n')
fw.write(r'\toprule' + '\n')

fw.write(r'& {Bench    } & {Sunk-cost} & {Sunk-cost} & {Sunk-cost} & {Reentry  } & {Search   } & {Starters } & {No-Cost  } \\' + '\n') 
fw.write(r'& {       } & {       } & {High   } & {Iceberg} & {       } & {       } & {       } & {       } \\' + '\n') 
fw.write(r'\midrule'+ '\n') 

fw.write(r'\multicolumn{9}{l}{\textit{Trade Elasticity}} \\ [0.5ex]' + '\n')
fw.write(format_row('Discounted', 'delas', dis_changes, models, '0.2f' ))
fw.write(format_row('Steady state', 'elast', ss_changes, models, '0.2f' ))

fw.write(r'[1.5ex]\multicolumn{9}{l}{\textit{Change in}} \\ [0.5ex]' + '\n')
fw.write(format_row('Welfare', 'dwelfare', dis_changes, models, '0.2f' ))
fw.write(format_row('Consumption', 'C', ss_changes, models, '0.2f' ))
fw.write(format_row('Estab.', 'NT', ss_changes, models, '0.2f' ))
fw.write(format_row('Exporters', 'n_X', ss_changes, models, '0.2f' ))
fw.write(format_row('Ex. Intensity', 'eei', ss_changes, models, '0.2f' ))
fw.write(format_row('Ex. Premium', 'prem', ss_changes, models, '0.2f' ))
fw.write(format_row('Iceberg Cost', 'ice', ss_changes, models, '0.2f' ))


fw.write(r'[1.5ex]\multicolumn{9}{l}{\textit{SS output decomposition (Discounted values in brackets)}} \\ [0.5ex]' + '\n')
fw.write(format_row(r'$\widehat{Y}$', 'Y', ss_changes, models, '0.2f' ))
fw.write(format_row_dis(r'', 'dy', dis_changes, models, '0.2f' ))

fw.write(format_row(r'$\widehat{L}_p$', 'LP', ss_changes, models, '0.2f' ))
fw.write(format_row_dis(r'', 'dlp', dis_changes, models, '0.2f' ))

fw.write(format_row(r'$\alpha \widehat{\frac{K}{L}}$', 'alphaKL', ss_changes, models, '0.2f' ))
fw.write(format_row_dis(r'', 'dkl', dis_changes, models, '0.2f' ))

fw.write(format_row(r'$\widehat{S-\frac{\alpha_x(\theta-1)}{\theta}}$', 's_adj', ss_changes, models, '0.2f' ))
fw.write(format_row_dis(r'', 'dsadj', dis_changes, models, '0.2f' ))

fw.write(format_row(r'$\widehat{N^\dagger}$', 'nhat_norm', ss_changes, models, '0.2f' ))
fw.write(format_row_dis(r'', 'dn', dis_changes, models, '0.2f' ))

fw.write(format_row(r'$\widehat{\psi_d^\dagger}$', 'psidhat_norm', ss_changes, models, '0.2f' ))
fw.write(format_row_dis(r'', 'dpsid', dis_changes, models, '0.2f' ))

fw.write(format_row(r'$\widehat{\lambda^\dagger}$', 'lambdahat_norm', ss_changes, models, '0.2f' ))
fw.write(format_row_dis(r'', 'dlam', dis_changes, models, '0.2f' ))


fw.write(r'\bottomrule' + '\n')
fw.write(r'\end{tabular}' + '\n')
fw.write(r'\end{table}'+'\n')

fw.write(r'\end{document}')
fw.close()

Done.